In [50]:
import pandas as pd

ctgan_synthetic_data = []
import os
for dirname, _, filenames in os.walk("..\\thesisgan\\output\\ctabgan_sweep_run_1\\"):
    for filename in filenames:
        if 'syn' in filename:
            fx = os.path.join(dirname, filename)
            ctgan_synthetic_data.append(fx)            
            print(fx)

..\thesisgan\output\ctabgan_sweep_run_1\0\syn.csv
..\thesisgan\output\ctabgan_sweep_run_1\1\syn.csv
..\thesisgan\output\ctabgan_sweep_run_1\2\syn.csv
..\thesisgan\output\ctabgan_sweep_run_1\3\syn.csv
..\thesisgan\output\ctabgan_sweep_run_1\4\syn.csv
..\thesisgan\output\ctabgan_sweep_run_1\5\syn.csv
..\thesisgan\output\ctabgan_sweep_run_1\6\syn.csv
..\thesisgan\output\ctabgan_sweep_run_1\7\syn.csv


In [7]:
real_data = pd.read_csv("..\\thesisgan\\input\\new_hpo_data.csv")
test_data = pd.read_csv("..\\thesisgan\\input\\new_test_data.csv")
import sys
sys.path.append("..")
from ctabganplus.model.evaluation import get_utility_metrics

In [8]:
from sklearn.preprocessing import LabelEncoder

In [9]:
le_dict = {"attack_type": "le_attack_type", "label": "le_label", "proto": "le_proto", "tos": "le_tos"}
for c in le_dict.keys():
    le_dict[c] = LabelEncoder()
    test_data[c] = le_dict[c].fit_transform(test_data[c])
    real_data[c] = le_dict[c].fit_transform(real_data[c])

In [10]:
for synthetic_data in ctgan_synthetic_data:
    df = pd.read_csv(synthetic_data)
    for c in le_dict.keys():
        df[c] = le_dict[c].fit_transform(df[c])
    #if the synthetic data has only one unique value for the attack type, add a row with the attack type
    sample_data_value_counts = df["attack_type"].value_counts()
    for i in range(len(sample_data_value_counts)):
        if sample_data_value_counts[i] == 1:
            at = sample_data_value_counts.index[i]
            df = pd.concat([df,real_data[real_data["attack_type"] == at].sample(3)], ignore_index=True)
    for at in test_data["attack_type"].unique():
        if at not in df["attack_type"].unique():
        #add a row with the attack type
            df = pd.concat([df,real_data[real_data["attack_type"] == at].sample(3)], ignore_index=True)

    result_df, cr = get_utility_metrics(real_data, test_data, df, scaler="MinMax",type={"Classification":["xgb","lr","dt","rf","mlp"]})
    result_df["id"] = synthetic_data.split("\\")[4]
    cr["id"] = synthetic_data.split("\\")[4]
    path = synthetic_data.split("syn")[0]
    print(path)
    result_df.to_csv(path + "new_classification_results.csv", index=False)
    cr.to_csv(path + "new_classification_report.csv", index=False)
    print("Done for " + synthetic_data)

Model:  xgb trained on real data
Model:  lr trained on real data
Model:  dt trained on real data
Model:  rf trained on real data
Model:  mlp trained on real data
Model:  xgb trained on fake data
Model:  lr trained on fake data
Model:  dt trained on fake data
Model:  rf trained on fake data
Model:  mlp trained on fake data
..\thesisgan\output\ctabgan_sweep_run_1\0\
Done for ..\thesisgan\output\ctabgan_sweep_run_1\0\syn.csv
Model:  xgb trained on real data
Model:  lr trained on real data
Model:  dt trained on real data
Model:  rf trained on real data
Model:  mlp trained on real data
Model:  xgb trained on fake data
Model:  lr trained on fake data
Model:  dt trained on fake data
Model:  rf trained on fake data
Model:  mlp trained on fake data
..\thesisgan\output\ctabgan_sweep_run_1\1\
Done for ..\thesisgan\output\ctabgan_sweep_run_1\1\syn.csv
Model:  xgb trained on real data
Model:  lr trained on real data
Model:  dt trained on real data
Model:  rf trained on real data
Model:  mlp trained

# HPO Results Evaluation

In [66]:
import pandas as pd

ctabgan_classification_results = []
ctabgan_classification_report = []
import os
for dirname, _, filenames in os.walk("..\\thesisgan\\output\\ctabgan_sweep_run\\"):
    for filename in filenames:
        if "new_classification_report" in filename:
            fx = os.path.join(dirname, filename)
            ctabgan_classification_report.append(fx)            
            print(fx)
        elif "new_classification_results" in filename:
            fx = os.path.join(dirname, filename)
            ctabgan_classification_results.append(fx)
            print(fx)

..\thesisgan\output\ctabgan_sweep_run\0\new_classification_report.csv
..\thesisgan\output\ctabgan_sweep_run\0\new_classification_results.csv
..\thesisgan\output\ctabgan_sweep_run\1\new_classification_report.csv
..\thesisgan\output\ctabgan_sweep_run\1\new_classification_results.csv
..\thesisgan\output\ctabgan_sweep_run\2\new_classification_report.csv
..\thesisgan\output\ctabgan_sweep_run\2\new_classification_results.csv
..\thesisgan\output\ctabgan_sweep_run\3\new_classification_report.csv
..\thesisgan\output\ctabgan_sweep_run\3\new_classification_results.csv
..\thesisgan\output\ctabgan_sweep_run\4\new_classification_report.csv
..\thesisgan\output\ctabgan_sweep_run\4\new_classification_results.csv
..\thesisgan\output\ctabgan_sweep_run\5\new_classification_report.csv
..\thesisgan\output\ctabgan_sweep_run\5\new_classification_results.csv
..\thesisgan\output\ctabgan_sweep_run\6\new_classification_report.csv
..\thesisgan\output\ctabgan_sweep_run\6\new_classification_results.csv
..\thesisgan\

In [ ]:
import json

In [67]:
# read every json file into a dataframe and append to a main dataframe
# before appending, add the id j0cor82r from the path of the file offline-run-20240509_081254-j0cor82r for each
for file in ctabgan_classification_report:
    cr_df = pd.read_csv(file)
    cr_df["id"] = file.split("\\")[4]
    cr_df["run"] = "2"
    ctabgan_classification_report_df = pd.concat([ctabgan_classification_report_df, cr_df])
    ctabgan_classification_report_df.reset_index(drop=True, inplace=True)

In [71]:
ctabgan_classification_report_df

,attack_type,precision,recall,f1-score,support,Model,type,id,run
0,benign,1.00,1.00,1.00,261660,xgb,real,0,1
1,bruteForce,0.00,0.00,0.00,253,xgb,real,0,1
2,dos,1.00,1.00,1.00,62741,xgb,real,0,1
3,pingScan,0.36,0.57,0.44,7,xgb,real,0,1
4,portScan,0.95,0.99,0.97,2630,xgb,real,0,1
5,accuracy,0.00,0.00,1.00,327291,xgb,real,0,1
6,macro,0.66,0.71,0.68,327291,xgb,real,0,1
7,weighted,1.00,1.00,1.00,327291,xgb,real,0,1
8,benign,1.00,0.44,0.61,261660,lr,real,0,1
9,bruteForce,0.00,0.00,0.00,253,lr,real,0,1


In [22]:
pd.set_option("display.max_rows", None)

### Classification Results

In [73]:
for file in ctabgan_classification_results:
    cr_df = pd.read_csv(file)
    cr_df["id"] = file.split("\\")[4]
    cr_df["run"] = "1"
    ctabgan_classification_results_df = pd.concat([ctabgan_classification_results_df, cr_df])
    ctabgan_classification_results_df.reset_index(drop=True, inplace=True)
    

In [72]:
5*3*10

150

In [88]:
ctabgan_classification_results_df

,Acc,AUC,F1_Score,Model,Type,id,run
0,71.263699,0.965384,0.682713,xgb,Real,0,1
1,50.098031,0.723246,0.336102,lr,Real,0,1
2,70.680787,0.816690,0.681611,dt,Real,0,1
3,71.364228,0.947653,0.688771,rf,Real,0,1
4,59.509164,0.864378,0.592110,mlp,Real,0,1
5,59.600926,0.842736,0.516676,xgb,Fake,0,1
6,57.048639,0.707772,0.462528,lr,Fake,0,1
7,39.142070,0.619638,0.393553,dt,Fake,0,1
8,48.565879,0.800677,0.519201,rf,Fake,0,1
9,54.285714,0.824479,0.400380,mlp,Fake,0,1


In [93]:
ctabgan_classification_results_df[(ctabgan_classification_results_df["Type"] == "Fake") & (ctabgan_classification_results_df["Model"] != "lr")].sort_values(by="F1_Score", ascending=True)


,Acc,AUC,F1_Score,Model,Type,id,run
217,3.506696,0.396917,0.055228,dt,Fake,6,1
215,29.526456,0.611488,0.070655,xgb,Fake,6,1
260,10.681783,0.480209,0.123675,xgb,Fake,9,1
219,26.131177,0.565900,0.167571,mlp,Fake,6,1
262,19.817858,0.498862,0.179543,dt,Fake,9,1
263,24.289796,0.602864,0.181881,rf,Fake,9,1
97,39.189718,0.619936,0.188395,dt,Fake,6,1
99,44.008194,0.681676,0.201758,mlp,Fake,6,1
264,38.318576,0.650401,0.216604,mlp,Fake,9,1
22,28.429452,0.552684,0.297031,dt,Fake,1,1


In [106]:
ctabgan_classification_results_df.drop(["Model"],axis=1).groupby(["Type","run","id"]).mean().sort_values(by="F1_Score", ascending=False).head(100)


Acc       AUC  F1_Score
Type       run id                               
Real       1   9   64.583182  0.863470  0.596262
               8   64.583182  0.863470  0.596262
               7   64.583182  0.863470  0.596262
               6   64.583182  0.863470  0.596262
               5   64.583182  0.863470  0.596262
               4   64.583182  0.863470  0.596262
               3   64.583182  0.863470  0.596262
               2   64.583182  0.863470  0.596262
               1   64.583182  0.863470  0.596262
               0   64.583182  0.863470  0.596262
Fake       1   7   54.297317  0.809072  0.530441
               8   52.534486  0.786067  0.520069
               4   54.267046  0.777935  0.519975
               5   53.771900  0.794550  0.519934
               0   51.892250  0.748355  0.486159
               3   48.484988  0.768120  0.468126
Difference 1   9   41.026456  0.302282  0.450465
Fake       1   1   47.656836  0.740408  0.441930
               2   49.514178  0.772503  0.437996
Difference 1   6   29.223527  0.233000  0.389746
Fake       1   6   35.359655  0.630470  0.206515
Difference 1   2   15.069004  0.090967  0.158265
               1   16.926346  0.123063  0.154331
Fake       1   9   23.556726  0.561189  0.145796
Difference 1   3   16.098194  0.095351  0.128135
               0   12.690932  0.115115  0.110102
               5   10.811282  0.068920  0.076328
               4   10.316136  0.085535  0.076287
               8   12.048696  0.077403  0.076192
               7   10.285865  0.054398  0.065820

# HPO Results Evaluation

In [ ]:
import pandas as pd

ctgan_classification_report = []
ctgan_classification_results = []
import os
for dirname, _, filenames in os.walk("..\\thesisgan\\output\\CTGAN_HPO\\"):
    for filename in filenames:
        if "new_classification_report" in filename:
            fx = os.path.join(dirname, filename)
            ctgan_classification_report.append(fx)            
            print(fx)
        elif "new_classification_results" in filename:
            fx = os.path.join(dirname, filename)
            ctgan_classification_results.append(fx)
            print(fx)

..\thesisgan\output\CTGAN_HPO\0\new_classification_report.csv
..\thesisgan\output\CTGAN_HPO\0\new_classification_results.csv
..\thesisgan\output\CTGAN_HPO\1\new_classification_report.csv
..\thesisgan\output\CTGAN_HPO\1\new_classification_results.csv
..\thesisgan\output\CTGAN_HPO\10\new_classification_report.csv
..\thesisgan\output\CTGAN_HPO\10\new_classification_results.csv
..\thesisgan\output\CTGAN_HPO\11\new_classification_report.csv
..\thesisgan\output\CTGAN_HPO\11\new_classification_results.csv
..\thesisgan\output\CTGAN_HPO\12\new_classification_report.csv
..\thesisgan\output\CTGAN_HPO\12\new_classification_results.csv
..\thesisgan\output\CTGAN_HPO\13\new_classification_report.csv
..\thesisgan\output\CTGAN_HPO\13\new_classification_results.csv
..\thesisgan\output\CTGAN_HPO\14\new_classification_report.csv
..\thesisgan\output\CTGAN_HPO\14\new_classification_results.csv
..\thesisgan\output\CTGAN_HPO\15\new_classification_report.csv
..\thesisgan\output\CTGAN_HPO\15\new_classification_

In [ ]:
import json

In [ ]:
# read every json file into a dataframe and append to a main dataframe
# before appending, add the id j0cor82r from the path of the file offline-run-20240509_081254-j0cor82r for each

ctgan_classification_report_df = pd.DataFrame()
for file in ctgan_classification_report:
    cr_df = pd.read_csv(file)
    cr_df["id"] = file.split("\\")[4]
    ctgan_classification_report_df = pd.concat([ctgan_classification_report_df, cr_df])
    ctgan_classification_report_df.reset_index(drop=True, inplace=True)

In [ ]:
pd.set_option("display.max_rows", None)

### Classification Results

In [ ]:
ctgan_classification_results_df = pd.DataFrame()
for file in ctgan_classification_results:
    cr_df = pd.read_csv(file)
    cr_df["id"] = file.split("\\")[4]
    ctgan_classification_results_df = pd.concat([ctgan_classification_results_df, cr_df])
    ctgan_classification_results_df.reset_index(drop=True, inplace=True)

In [ ]:
ctgan_classification_results_df[ctgan_classification_results_df["Type"] == "Fake"].sort_values(by="F1_Score", ascending=False).sort_values(by="id", ascending=False).head(100)

,Acc,AUC,F1_Score,Model,Type,id
234,73.168639,0.888510,0.676596,rf,Fake,9
233,57.923670,0.737023,0.389336,dt,Fake,9
232,64.025036,0.836684,0.589617,lr,Fake,9
235,71.790143,0.924916,0.686023,mlp,Fake,9
221,33.358948,0.583493,0.278108,dt,Fake,8
222,49.351912,0.838198,0.505383,rf,Fake,8
223,49.215455,0.857012,0.523632,mlp,Fake,8
220,40.383374,0.760958,0.405967,lr,Fake,8
209,35.258814,0.595368,0.301022,dt,Fake,7
208,50.006385,0.697204,0.525267,lr,Fake,7


In [ ]:
ctgan_classification_results_df.drop(["Model"],axis=1).groupby(["id", "Type"]).mean().sort_values(by="F1_Score", ascending=False).head(100)


,,Acc,AUC,F1_Score
id,Type,,,
9,Real,62.932143,0.855187,0.629673
14,Real,62.932143,0.855187,0.629673
15,Real,62.932143,0.855187,0.629673
13,Real,62.932143,0.855187,0.629673
6,Real,62.932143,0.855187,0.629673
16,Real,62.932143,0.855187,0.629673
12,Real,62.932143,0.855187,0.629673
17,Real,62.932143,0.855187,0.629673
8,Real,62.932143,0.855187,0.629673
